In [6]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain_community wikipedia

  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/7f/9a/accade34b8668b21206c0c4cf0b96cd0b750b693ba5b255c1c10cfee460f/faiss_cpu-1.11.0-cp311-cp311-win_amd64.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
    --------------------------------------- 0.3/15.0 MB 9.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.7/15.0 MB 21.3 MB/s eta 0:00:01
   ---------- ----------------------------- 3.8/15.0 MB 30.2 MB/s eta 0:00:01
   --------------- ------------------------ 5.8/15.0 MB 33.6 MB/s eta 0:00:01
   -------------------- ------------------- 7.7/15.0 MB 35.1 MB/s eta 0:00:01
   ------------------------- -------------- 9.6/15.0 MB 36.3 MB/s eta 0:00:01
   ------------------------------- -------- 11.6/15.0 MB 40.9 MB/s eta 0:00:01
   ------------------------------------ --- 13.6/15.0 MB 40.9 MB/s et

In [4]:
#Wikipedia Retriever
from langchain_community.retrievers import WikipediaRetriever

In [7]:
retriever = WikipediaRetriever(top_k_results=2, lang="en")
query = "The landscape of Norway  and Ireland and the scottish background"
docs = retriever.invoke(query)

In [8]:
for i, doc in enumerate(docs):
    print(f"\n --- Result {i+1}---")
    print(f"Content: \n{doc.page_content}.\n")


 --- Result 1---
Content: 
Scotland is a country that is part of the United Kingdom. It contains nearly one-third of the United Kingdom's land area, consisting of the northern part of the island of Great Britain and more than 790 adjacent islands, principally in the archipelagos of the Hebrides and the Northern Isles. To the south-east, Scotland has its only land border, which is 96 miles (154 km) long and shared with England; the country is surrounded by the Atlantic Ocean to the north and west, the North Sea to the north-east and east, and the Irish Sea to the south. The population in 2022 was 5,439,842. Edinburgh is the capital and Glasgow is the most populous of the cities of Scotland.
The Kingdom of Scotland emerged as an independent sovereign state in the 9th century. In 1603, James VI succeeded to the thrones of England and Ireland, forming a personal union of the three kingdoms. On 1 May 1707, Scotland and England combined to create the new Kingdom of Great Britain, with the P

In [9]:
#Vector Store Retriever
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [10]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [12]:
embedding_model = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding = embedding_model,
    collection_name= "my_collection"
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [13]:
retriever = vectorstore.as_retriever(search_kwargs={"k":2})

In [14]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [15]:
for i,doc in enumerate(results):
    print(f"\n --- Results{i+1} ---")
    print(doc.page_content)


 --- Results1 ---
Chroma is a vector database optimized for LLM-based search.

 --- Results2 ---
LangChain helps developers build LLM applications easily.


In [16]:
#MMR
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [17]:
from langchain_community.vectorstores import FAISS

embedding_model = OpenAIEmbeddings()
vectorestore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

retriever = vectorstore.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k":3, "lambda_mult":0.5}
)

In [18]:
query = "What is langchain ?"
results = retriever.invoke(query)

In [19]:
for i, doc in enumerate(results):
    print(f"\n --- Results{i+1} ---")
    print(doc.page_content)


 --- Results1 ---
LangChain helps developers build LLM applications easily.

 --- Results2 ---
Chroma is a vector database optimized for LLM-based search.

 --- Results3 ---
Embeddings convert text into high-dimensional vectors.


In [20]:
##Multiquery Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [21]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [25]:
embedding_model = OpenAIEmbeddings()
vectore_store = FAISS.from_documents(documents=all_docs, embedding=embedding_model)
similarity_retriever = vectore_store.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [26]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectore_store.as_retriever(search_kwargs={"k":5}),
    llm = ChatOpenAI(model="gtp-3.5-turbo")
)
query = "How to improve energy levels and maintain balance?"

In [ ]:
similarity_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

In [ ]:
for i,doc in enumerate(similarity_results):
    print(f"\n --- Result{i+1} ---")
    print(doc.page_content)

print("*"150)

for i,doc in enumerate(multiquery_results):
    print(f"\n --- Result{i+1} ---")
    print(doc.page_content)


In [ ]:
#ContextualCompressionRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [ ]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [ ]:
embedding_model = OpenAIEmbeddings()
vectore_store = FAISS.from_documents(docs, embedding_model)
base_retriever = vectore_store.as_retriever(search_kwargs={"k":5})

In [ ]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo')
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [ ]:
query = 'What is photosynthesis?'
compressed_results = compression_retriever.invoke(query)
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)